In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import h5py

def DL1_score(pb, pc, pl): 
    return np.log(pb/(0.018*pc + 0.982*pl))

def plot_prob_score(pb, pc, pl):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    _n, bins, _ = ax1.hist(pb, bins=50, range=(0,1), alpha=0.5, label="p_b")
    _ = ax1.hist(pc, bins=bins, alpha=0.5, label="p_c")
    _ = ax1.hist(pl, bins=bins, alpha=0.5, label="p_l")
    ax1.legend(loc="upper right")
    
    _ =ax2.hist(DL1_socre(pb, pc, pl), 100, alpha=0.6)

def plot_prob_score_from_model(event, model):
    
    Single_Pred_prob = model(np.array(10000*[X_test[event,:]]))
    pb = Single_Pred_prob[:,2].numpy()
    pc = Single_Pred_prob[:,1].numpy()
    pl = Single_Pred_prob[:,0].numpy()
    print("true label: ", Y_test[event])
    print("softmax prob: ", Single_Pred_prob[event])
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    _n, bins, _ = ax1.hist(pb, bins=50, range=(0,1),  alpha=0.5, label="p_b")
    _ = ax1.hist(pc, bins=bins,  alpha=0.5, label="p_c")
    _ = ax1.hist(pl, bins=bins,  alpha=0.5, label="p_l")
    ax1.legend(loc="upper right")
    
    _ =ax2.hist(DL1_socre(pb, pc, pl), 100,  alpha=0.6)

def efficiency(score, wp=1.0):
    return np.sum(score>wp)/len(score)
    
    
hf = h5py.File("/Users/abletimin/cernbox/b-tagg/btagging-ml_tutorial_files/MC16d_ttbar-test-validation_sample-NN.h5", 'r')
print(hf.keys())
X_test, Y_test = hf['X_test'][:], hf['Y_test'][:]
hf.close() 
print("Total number of events: ",len(X_test) )

<KeysViewHDF5 ['X_test', 'X_trk_test', 'Y_test', 'labels', 'pt_eta']>
Total number of events:  1928080


## import model


In [18]:
DL1_model = tf.keras.models.load_model("/Users/abletimin/cernbox/b-tagg/trainedModel/DL1r_2M_300epoch_3600batch.h5")
DL1_model.summary()

Pred_prob = DL1_model(X_test)

#print("true label: ", Y_test[event])
acc = tf.keras.metrics.CategoricalAccuracy()
acc.update_state(Y_test, Pred_prob.numpy())
print ("Acc: ", acc.result().numpy())

acc.update_state(Y_test[:,2], Pred_prob[:2])
print ("bjet Acc: ", acc.result().numpy())
acc.update_state(Y_test[:,1], Pred_prob[:1])
print ("cjet Acc: ", acc.result().numpy())
acc.update_state(Y_test[:,0], Pred_prob[:0])
print ("ljet Acc: ", acc.result().numpy())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 78)                3510      
_________________________________________________________________
activation (Activation)      (None, 78)                0         
_________________________________________________________________
dropout (Dropout)            (None, 78)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 78)                312       
_________________________________________________________________
dense_1 (Dense)              (None, 66)                5214      
_________________________________________________________________
dropout_1 (Dropout)          (None, 66)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 66)                2

In [19]:
Pred_prob_pb = Pred_prob[Y_test[:,2]==1]
efficiency( DL1_score(Pred_prob_pb[:,2], Pred_prob_pb[:,1], Pred_prob_pb[:,0]), 1.45)

0.7978849744952022

In [15]:
Pred_prob_pb[:,2]

<tf.Tensor: shape=(750251,), dtype=float32, numpy=
array([0.9958332 , 0.37944305, 0.24621114, ..., 0.19877161, 0.7958348 ,
       0.12091827], dtype=float32)>

In [2]:
hfile = h5py.File("/Users/abletimin/cernbox/b-tagg/trainedModel/DL1r_2M_300epoch_3600batch.h5")

<ipython-input-2-27db51e43769>:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  hfile = h5py.File("/Users/abletimin/cernbox/b-tagg/trainedModel/DL1r_2M_300epoch_3600batch.h5")


In [10]:
hfile["model_weights"].keys()

<KeysViewHDF5 ['activation', 'batch_normalization', 'batch_normalization_1', 'batch_normalization_2', 'batch_normalization_3', 'batch_normalization_4', 'batch_normalization_5', 'batch_normalization_6', 'batch_normalization_7', 'dense', 'dense_1', 'dense_2', 'dense_3', 'dense_4', 'dense_5', 'dense_6', 'dense_7', 'dense_8', 'dropout', 'dropout_1', 'dropout_2', 'dropout_3', 'dropout_4', 'dropout_5', 'dropout_6', 'dropout_7']>